<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Recommendation-Engine" data-toc-modified-id="Recommendation-Engine-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Recommendation Engine</a></span><ul class="toc-item"><li><span><a href="#Distance-functions" data-toc-modified-id="Distance-functions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Distance functions</a></span></li><li><span><a href="#Nearest-neighbors" data-toc-modified-id="Nearest-neighbors-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Nearest neighbors</a></span></li><li><span><a href="#Recommendation-function" data-toc-modified-id="Recommendation-function-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Recommendation function</a></span></li><li><span><a href="#Weighted-k-nearest-neighbors" data-toc-modified-id="Weighted-k-nearest-neighbors-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Weighted k nearest neighbors</a></span></li></ul></li><li><span><a href="#Book-recommendations---Real-world-example" data-toc-modified-id="Book-recommendations---Real-world-example-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Book recommendations - Real world example</a></span><ul class="toc-item"><li><span><a href="#Environment" data-toc-modified-id="Environment-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Environment</a></span></li><li><span><a href="#Wrangle-dataset" data-toc-modified-id="Wrangle-dataset-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Wrangle dataset</a></span></li><li><span><a href="#Test-recommendation-engine" data-toc-modified-id="Test-recommendation-engine-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Test recommendation engine</a></span></li></ul></li></ul></div>

# Recommendation Engine

This is my learnings from following [A Programmer's Guide to Data Mining](http://guidetodatamining.com/), warmly recommended.

The first goal of this project is to figure out who is similar to who.
We start with some dummy data, these are users with a dictionary of titles and rating.

In [1]:
users = {
    "Angelica": {"Blues Traveler": 3.5, "Broken Bells": 2.0, "Norah Jones": 4.5, "Phoenix": 5.0, "Slightly Stoopid": 1.5, "The Strokes": 2.5, "Vampire Weekend": 2.0},
    "Bill": {"Blues Traveler": 2.0, "Broken Bells": 3.5, "Deadmau5": 4.0, "Phoenix": 2.0, "Slightly Stoopid": 3.5, "Vampire Weekend": 3.0},
    "Chan": {"Blues Traveler": 5.0, "Broken Bells": 1.0, "Deadmau5": 1.0, "Norah Jones": 3.0, "Phoenix": 5, "Slightly Stoopid": 1.0},
    "Dan": {"Blues Traveler": 3.0, "Broken Bells": 4.0, "Deadmau5": 4.5, "Phoenix": 3.0, "Slightly Stoopid": 4.5, "The Strokes": 4.0, "Vampire Weekend": 2.0},
    "Hailey": {"Broken Bells": 4.0, "Deadmau5": 1.0, "Norah Jones": 4.0, "The Strokes": 4.0, "Vampire Weekend": 1.0},
    "Jordyn":  {"Broken Bells": 4.5, "Deadmau5": 4.0, "Norah Jones": 5.0, "Phoenix": 5.0, "Slightly Stoopid": 4.5, "The Strokes": 4.0, "Vampire Weekend": 4.0},
    "Sam": {"Blues Traveler": 5.0, "Broken Bells": 2.0, "Norah Jones": 3.0, "Phoenix": 5.0, "Slightly Stoopid": 4.0, "The Strokes": 5.0},
    "Veronica": {"Blues Traveler": 3.0, "Norah Jones": 5.0, "Phoenix": 4.0, "Slightly Stoopid": 2.5, "The Strokes": 3.0}
}

## Distance functions
Let's add some typical distance functions: 
[Minkowski distance](https://en.wikipedia.org/wiki/Minkowski_distance), 
[Pearson correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient),
and [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity).
Note that the latter two needs to be negated to act as a distance!

In [2]:
def minkowski_distance(a, b, p=1):
    """
    Get distance between overlapping members of dicts a and b
    using minkowski distance of power p
    p=1 manhattan
    p=2 euclidean
    """
    return sum(abs(a[key] - b[key])**p for key in set(a.keys()).intersection(b.keys()))**(1./p)

In [3]:
def pearson_correlation(a, b):
    """
    Get correlation between overlapping members of dicts a and b
    """
    keys = set(a.keys()).intersection(b.keys())
    n = float(len(keys))
    if n == 0:
        return 0
    x = [a[k] for k in keys]
    y = [b[k] for k in keys]
    sx = sum(x)
    sy = sum(y)
    sx2 = sum(map(lambda x: x * x, x))
    sy2 = sum(map(lambda x: x * x, y))
    d = ((sx2 - sx * sx / n) * (sy2 - sy * sy / n)) ** 0.5
    if d == 0:
        return 0
    sxy = sum(map(lambda x, y: x * y, x, y))
    return (sxy - sx * sy / n) / d

In [4]:
def cosine_similarity(a, b):
    """
    Get cosine simuliarty between overlapping members of dicts a and b
    a dot b / (|a|*|b|)
    """
    keys = set(a.keys()).intersection(b.keys())
    x = [a[k] for k in keys]
    y = [b[k] for k in keys]
    ax = sum(map(lambda x: x * x, x)) ** 0.5
    ay = sum(map(lambda x: x * x, y)) ** 0.5
    sxy = sum(map(lambda x, y: x * y, x, y))
    return sxy / (ax * ay)

In [5]:
# Convenience dictionary
DISTANCES = {
    "Manhattan": lambda x, y: minkowski_distance(x, y, p=1),
    "Euclidean": lambda x, y: minkowski_distance(x, y, p=2),
    "Pearson": lambda x, y: 1-pearson_correlation(x, y)*0.5,  # new range [0,2]
    "Cosine": lambda x, y: 1-cosine_similarity(x, y)  # new range [0,1]
}

## Nearest neighbors
To see how close a user `who` is to any other user we compute the distance sorted nearest neighbors.

In [6]:
def nearest_neighbors(who, users, distance=DISTANCES["Manhattan"]):
    """
    Get list of neighbors in users sorted by distance
    """
    # Get list of (user, distance) tuples for all other users
    neighbors = [
        (user, distance(users[who], scores))
        for user, scores in users.items() if user != who]
    # Sort list inplace and return it
    neighbors.sort(key=lambda x: x[1])
    return neighbors

Let's use this to print the `3` closest users to `Bill` using several different distance estimators.

In [7]:
for d, distance in DISTANCES.items():
    print(d, nearest_neighbors("Angelica", users, distance=distance)[:3])

Manhattan [('Veronica', 3.5), ('Chan', 4.5), ('Hailey', 5.0)]
Euclidean [('Veronica', 1.6583123951777), ('Chan', 2.3979157616563596), ('Hailey', 2.7386127875258306)]
Pearson [('Veronica', 0.585307011886813), ('Chan', 0.5901088526350293), ('Jordyn', 0.6180125697262284)]
Cosine [('Veronica', 0.020936396109991096), ('Chan', 0.04586032801208695), ('Jordyn', 0.06580649324893406)]


The various methods actually look quite consistent. 
## Recommendation function
Now we can make a recommendation by finding the titles not in common with nearest neighbor.

In [8]:
def recommend(user, users, distance=DISTANCES["Manhattan"]):
    """Give list of recommendations"""
    # first find nearest neighbor
    nearest = nearest_neighbors(user, users, distance=distance)[0][0]
    # Find ratings that username does not have in common with nearest
    recommendations = [
        (title, rating) for title, rating in users[nearest].items()
        if not title in users[user]]
    # Sort by rating and return
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations

Let's make recommendations for everybody. For now we stick to Pearson correlation distance.

In [9]:
for user in users.keys():
    print(user, recommend(user, users, distance=DISTANCES["Pearson"]))

Angelica []
Bill [('The Strokes', 4.0)]
Chan [('The Strokes', 2.5), ('Vampire Weekend', 2.0)]
Dan []
Hailey [('Phoenix', 5.0), ('Slightly Stoopid', 4.5)]
Jordyn [('Blues Traveler', 5.0)]
Sam [('Deadmau5', 1.0)]
Veronica [('Broken Bells', 2.0), ('Vampire Weekend', 2.0)]


So from before we know Bill was close to Dan, and the only artist that Dan has that Bill hasn't is `The Strokes` so it gets recommended. This approach appears to be working but it is a little simplistic. 

## Weighted k nearest neighbors

Instead of using the _nearest_ neighbor, it is better to use the nearest K neighbors and weight the rating by distance. This allows non overlapping entries from several users to enter the results, making it less likely to get empty recommendations.

In [10]:
def recommend_knn(user, users, distance=DISTANCES["Pearson"], k=3):
    # first find nearest neighbor
    nearest = nearest_neighbors(user, users, distance=distance)[:k]
    total_score = sum(s for u, s in nearest)
    weighted = [(t, s, r) for u, s in nearest for t, r in users[u].items() if t not in users[user]]
    result = {}
    for u, s in nearest:
        for t, r in users[u].items():
            if t not in users[user]:
                result[t] = result.get(t, 0) + s * r
    return [(k, v/total_score) for k, v in result.items()]

Let's try it for all users

In [11]:
for user in users.keys():
    print(user, recommend_knn(user, users, distance=DISTANCES["Pearson"], k=3))

Angelica [('Deadmau5', 1.707433133691678)]
Bill [('The Strokes', 4.0), ('Norah Jones', 3.5602724263661094)]
Chan [('The Strokes', 3.851226315901307), ('Vampire Weekend', 1.9807176874675074)]
Dan [('Norah Jones', 3.3333491927018044)]
Hailey [('Phoenix', 5.0), ('Slightly Stoopid', 2.2639282444695232), ('Blues Traveler', 2.9165257815053565)]
Jordyn [('Blues Traveler', 3.817271371299083)]
Sam [('Deadmau5', 0.6529843410632098), ('Vampire Weekend', 1.097976427537215)]
Veronica [('Broken Bells', 2.3398138623569578), ('Vampire Weekend', 1.7729935450797063), ('Deadmau5', 1.624560193517164)]


The resulting recommendations are now not empty for `Dan` and `Angelica`, and most recommendations are longer.

# Book recommendations - Real world example
So let's try to apply this simple recommendation engine to a proper dataset. We will be using the [Book Crossing](http://www.informatik.uni-freiburg.de/~cziegler/BX/) dataset, with a lot of book reviews.

## Environment
For this example we'll make use of more external libraries and helper functions.

In [12]:
from IPython.display import display, HTML 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
# Plotting specifics
%matplotlib inline
plt.style.use("seaborn-poster") # Better than default plotting style
plt.rcParams['figure.figsize'] = (12, 12) # Larger figures
sns.set_context("poster")

In [14]:
def DF_DISP(df):
    """Pretty print a dataframe"""
    display(HTML(df.to_html()))
def show_cols(name, df):
    """Show name, length, and list of columns of dataframe"""
    print("{}: {} {}".format(name, len(df.index), list(df.columns)))
def show_top_counts(name, df, col):
    """Print counts of most common values in col"""
    top = df.groupby([col])[["count"]].count().sort_values(by="count", ascending=False)
    print("Top {}(s) in {} of {} unique".format(col, name, len(top.index)))
    DF_DISP(top.head())

## Wrangle dataset
I downloaded the `csv` version of the data set and  manually converted the dataset into UTF-8 and gzipped it.
Malformed lines are dropped. These are usually due to `&nbsp;` or similar html that adds an extra `;` which otherwise is used as field separator.  Furthermore, duplicate entries are dropped as they don't add anything to the analysis. This treatment has no effect on ratings, very small effect on books, and moderate effect on users. The remaining clean dataset is big enough for what we want to do.

In [15]:
def read_csv(filename):
    df = pd.read_csv(
        filename, delimiter=";", quotechar="\"", error_bad_lines=False, low_memory=False, warn_bad_lines=False)
    df.dropna(inplace=True) # remove missing values
    df["count"] = 1 # add a count column    
    df.drop_duplicates(inplace=True)
    return df

In [16]:
bx_users = read_csv("BX-CSV-Dump/BX-Users.csv.gz")
bx_books = read_csv("BX-CSV-Dump/BX-Books.csv.gz")
bx_ratings = read_csv("BX-CSV-Dump/BX-Book-Ratings.csv.gz")

Ensure user Age is an int in range 5-100.

In [17]:
# Force age to be int
bx_users["Age"] = bx_users["Age"].astype(int)
# Restict range of age
bx_users = bx_users[(bx_users["Age"] > 5) & (bx_users["Age"] < 100)]

Ensure Book year is int in range 1920-2020, and drop URL columns.

In [18]:
# Force year as int
bx_books["Year-Of-Publication"] = bx_books["Year-Of-Publication"].astype(int)
# Restrict publication date
bf_books = bx_books[(bx_books["Year-Of-Publication"] > 1920)
                    & (bx_books["Year-Of-Publication"] < 2020)]
# Drop URL columns
bx_books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
              inplace=True, axis=1)

Ensure book rating is int and a real rating in range 1-10, that ratings have matching user id, and that books have matching rating.

In [19]:
# Force rating as int
bx_ratings["Book-Rating"] = bx_ratings["Book-Rating"].astype(int)
# Restrict to ratings 1-10
bx_ratings = bx_ratings[(bx_ratings["Book-Rating"] >= 1) & (bx_ratings["Book-Rating"] <= 10)]
# Remove any ratings without matching user id
bx_ratings = bx_ratings[bx_ratings["User-ID"].isin(bx_users["User-ID"])]
# Remove any books without any ratings and any rating without books
bx_books = bx_books[bx_books["ISBN"].isin(bx_ratings["ISBN"])]
bx_ratings = bx_ratings[bx_ratings["ISBN"].isin(bx_books["ISBN"])]

In [20]:
bx_books["ISBN"] = bx_books["ISBN"].astype(str)
bx_ratings["ISBN"] = bx_ratings["ISBN"].astype(str)

## Test recommendation engine

In [21]:
user_ratings = {
    g: dict(zip(r["ISBN"].values, r["Book-Rating"].values))
    for g, r in bx_ratings[["User-ID", "ISBN", "Book-Rating"]].groupby("User-ID")}

In [22]:
for i, user in enumerate(user_ratings.keys()):
    recommendation = recommend_knn(user, user_ratings, distance=DISTANCES["Pearson"], k=3)
    print(bx_users[bx_users["User-ID"]==user].iloc[0].values)
    for isbn, score in recommendation:
        print("\t", bx_books[bx_books["ISBN"]==isbn].iloc[0].values)
    if i > 10: # skip after 10 users
        break

[19 'weston, ,' 14 1]
	 ['0553582747' 'From the Corner of His Eye' 'Dean Koontz' 2001
 'Bantam Books' 1]
	 ['0440223571' 'This Year It Will Be Different: And Other Stories'
 'Maeve Binchy' 1997 'Dell' 1]
	 ['0440225701' 'The Street Lawyer' 'JOHN GRISHAM' 1999 'Dell' 1]
[42 'appleton, wisconsin, usa' 17 1]
	 ['0375759778' 'Prague : A Novel' 'ARTHUR PHILLIPS' 2003
 'Random House Trade Paperbacks' 1]
	 ['0440223571' 'This Year It Will Be Different: And Other Stories'
 'Maeve Binchy' 1997 'Dell' 1]
	 ['0440225701' 'The Street Lawyer' 'JOHN GRISHAM' 1999 'Dell' 1]
[44 'black mountain, north carolina, usa' 51 1]
	 ['0375759778' 'Prague : A Novel' 'ARTHUR PHILLIPS' 2003
 'Random House Trade Paperbacks' 1]
	 ['0553582747' 'From the Corner of His Eye' 'Dean Koontz' 2001
 'Bantam Books' 1]
	 ['0440225701' 'The Street Lawyer' 'JOHN GRISHAM' 1999 'Dell' 1]
[51 'renton, washington, usa' 34 1]
	 ['0375759778' 'Prague : A Novel' 'ARTHUR PHILLIPS' 2003
 'Random House Trade Paperbacks' 1]
	 ['055358274

So same method is directly applicable to real world data!